In [1]:
import keras
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
import os,cv2
%matplotlib inline

Using TensorFlow backend.


In [2]:
base_model = VGG19(weights='imagenet',include_top=False)
# 获取各层的输出：
layer_outputs = [layer.output for layer in base_model.layers[2:20]]
# 获取各层的名称：
layer_names = []
for layer in base_model.layers[2:20]:
    layer_names.append(layer.name)
print(layer_names)

['block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_conv4', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_conv4', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3']


In [3]:
model = Model(inputs=base_model.input, outputs=layer_outputs)

In [4]:
import pandas as pd
df=pd.read_csv('./fer2013.csv',header=0,encoding="utf-8")
features=df.iloc[:,:]['pixels']

In [5]:
labels=np.array(df.iloc[:,:]['emotion']).astype('float32')

In [7]:
result=[]
for feature in features:
    img=np.array(feature.split(" ")).astype('float32').reshape(48,48)/255
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    result.append(img)
features=np.array(result)
features.shape

(35887, 48, 48, 3)

In [46]:
def conj_martix(martix):
    result=[]
    width=martix.shape[0]
    height=martix.shape[1]
    nums=martix.shape[2]
    for num in range(nums):
        result.append(martix[:,:,num].flatten())
    return result

from numpy import nanmean

def downsample(myarr,factor,estimator=nanmean):
    """
    Downsample a 2D array by averaging over *factor* pixels in each axis.
    Crops upper edge if the shape is not a multiple of factor.
    This code is pure np and should be fast.
    keywords:
        estimator - default to mean.  You can downsample by summing or
            something else if you want a different estimator
            (e.g., downsampling error: you want to sum & divide by sqrt(n))
    """
    ys,xs = myarr.shape
    crarr = myarr[:ys-(ys % int(factor)),:xs-(xs % int(factor))]
    dsarr = estimator( np.concatenate([[crarr[i::factor,j::factor] 
        for i in range(factor)] 
        for j in range(factor)]), axis=0)
    return dsarr

def upper_gram(metrix):
    features=[]
    gram_len=metrix.shape[1]
    for row in range(gram_len):
        for clo in range(gram_len):
            clos=clo+row
            if(clos>gram_len-1):
                break
            features.append((metrix[row][row+clo]))
    return np.array(features)

def get_features(origin_features,layers):
    features_mertix=[]
    for index in range(origin_features.shape[0]):
        cnn_featues=model.predict(origin_features[index:index+1,:,:,:])
        layer_features=cnn_featues[layers][0]#获取第三层的输出
        conj_mertix=np.array(conj_martix(layer_features))#拼接矩阵
        gram_metrix=np.dot(conj_mertix, conj_mertix.T)#计算 gram 矩阵`
        feature=upper_gram(gram_metrix)#取 gram 矩阵的上半区
        features_mertix.append(feature)
    return np.array(features_mertix)

In [ ]:
features=get_features(features,17)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier as KNN
knn = KNN(n_neighbors=1)
scores = cross_val_score(knn, features, labels, cv=10)
print(scores.mean())

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
scores = cross_val_score(rf, features, labels, cv=10)
print(scores.mean())

In [ ]:
features.shape

In [ ]:
# features.to_csv('./fer2013-features-one-vec.csv')

### 可视化

In [8]:
features[0].shape

(48, 48, 3)

In [4]:
# plt.imshow(features[0])

In [10]:
base_model = VGG19(weights='imagenet',include_top=False)
# 获取各层的输出：
layer_outputs = [layer.output for layer in base_model.layers[2:20]]
# 获取各层的名称：
layer_names = []
for layer in base_model.layers[2:20]:
    layer_names.append(layer.name)
print(layer_names)

['block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_conv4', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_conv4', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3']


In [12]:
model = Model(inputs=base_model.input, outputs=layer_outputs)
# 将前面的图片数据x，输入到model中，得到各层的激活值activations：
activations = model.predict(features[0:1])

In [5]:
# import math
# for activation,layer_name in zip(activations[3:4],layer_names[3:4]):
#     h = activation.shape[1]
#     w = activation.shape[2]
#     num_channels = activation.shape[3]
#     cols = 3
#     rows = 1
#     img_grid = np.zeros((h*rows,w*cols))

#     for c in range(3):
#         f_r = math.ceil((c+1)/cols)
#         f_c = (c+1)if f_r==1 else (c+1-(f_r-1)*cols)
#         img_grid[(f_r-1)*h:f_r*h,(f_c-1)*w:f_c*w ] = activation[0,:,:,c]


#     plt.figure(figsize=(25,25))
#     plt.imshow(img_grid, aspect='equal',cmap='viridis')
#     plt.grid(False)
#     plt.title(layer_name,fontsize=16)
# plt.show()

In [ ]:
import cv2
filename = './e7.jpg'
img = cv2.imread(filename)